In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [4]:
import grader

# Mapreduce

## Introduciton

We are going to be running mapreduce jobs on the wikipedia dataset.  The dataset is available (pre-chunked) on S3: `s3://dataincubator-course/mrdata/simple/`.  It may be downloaded with the `aws s3 sync` command or via HTTPS from `https://s3.amazonaws.com/dataincubator-course/mrdata/simple/part-000*`.

For development, you can even use a single chunk (eg. part-00026.xml.bz2). That is small enough that mrjob can process the chunk in a few seconds. Your development cycle should be:

1.  Get your job to work locally on one chunk.  This will greatly speed up your
development.  To run on local:
```bash
python job_file.py -r local data/wikipedia/simple/part-00026.xml.bz2 > /tmp/output.txt
```
    
2.  Get your job to work on the full dataset on GCP (Google Cloud Platform).  This will greatly speed up your production.  To run on GCP ([details](https://pythonhosted.org/mrjob/guides/dataproc-quickstart.html)):
```bash
python job_file.py -r dataproc data/wikipedia/simple/part-00026.xml.bz2 \
    --output-dir=gs://my-bucket/output/ \
    --no-output 
```

    Not that you can also pass an entire local directory of data (eg. `data/simple/`) as the input.

### Note on Memory
There's a large difference between developing locally on one chunk and running your job on the entire dataset.  While you can get away with sloppy memory use locally, you really need to keep memory usage down if you hope to be able to complete the miniproject.  Remember, memory needs to be $O(1)$, not $O(n)$ in input.

### Multiple Mapreduces
You can combine multiple steps by overriding the [steps method](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs).  Usually your mapreduce might look like this
```python
from mrjob.job import MRJob

class SingleMRJob(MRJob):
    def mapper(self, key, value):
        pass

    def reducer(self, key, values):
        pass
```

`MRJob` automatically uses the `mapper` and `reducer` methods.  To specify multiple steps, you need to override the `steps` method:

```python
from mrjob.job import MRJob
from mrjob.step import MRStep

class MultipleMRJob(MRJob):
    def mapper1(self, key, value):
        pass

    def reducer1(self, key, values):
        pass
        
    def mapper2(self, key, value):
        pass

    def reducer2(self, key, values):
        pass
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2),
        ]
```

As a matter of good style, we recommend that you actually write each individual mapreduce as it's own class.  Then write a wrapper module whose sole job is to combine those mapreduces by overriding `steps`.

Some simple boilerplate for this, taking advantage of the default `steps` function that we get for free in a single-step MRJob class:

```python
class FirstStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SecondStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SteppedJob(MRJob):
  """
  A two-step job that first runs FirstStep's MR and then SecondStep's MR
  """
  def steps(self):
    return FirstStep().steps() + SecondStep().steps()
```


### Note on Style
Here are some helpful articles on how mrjob works and how to pass parameters to your script:
  - [How mrjob is run](https://pythonhosted.org/mrjob/guides/concepts.html#how-your-program-is-run)
  - [Adding passthrough options](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.add_passthrough_option)
  - [An example of someone solving similar problems](http://arunxjacob.blogspot.com/2013/11/hadoop-streaming-with-mrjob.html)

See the notebook "Hadoop MapReduce with mrjob" in the datacourse for more details.

Finally, if you are find yourself processing a lot of special cases, you are probably doing it wrong.  For example, mapreduce jobs for `Top100WordsSimpleWikipediaPlain`, `Top100WordsSimpleWikipediaText`, and `Top100WordsSimpleWikipediaNoMetaData` are less than 150 lines of code (including generous blank lines and biolerplate code).

## Question 1: top100_words_simple_plain
Return a list of the top 100 words in an article text (in no particular order). You will need to write this as two map reduces:

1. The first job is similar to standard wordcount but with a few tweaks. The data provided for wikipedia is in `*.xml.bz2` format.  Mrjob will automatically decompress `bz2`.  We'll deal with the `xml` in the next question. For now, just treat it as text.  A few hints:
   - To split the words, use the regular expression "\w+".
   - Words are not case sensitive: i.e. "The" and "the" reference to the same word.  You can use `string.lower()` to get a single case-insenstive canonical version of the data.

2. The second job will take a collection of pairs `(word, count)` and filter for only the highest 100.  A few notes:
    - **Passing parameters:** To make the job more reusable make the job find the largest `n` words where `n` is a parameter obtained via [`get_jobconf_value`](https://pythonhosted.org/mrjob/utils-compat.html).
    - **Keeping track of the top n:** We have to keep track of at most the `n` most popular words.  As long as `n` is small, e.g. 100, we can keep track of the *running largest n* in memory wtih a priority-queue. We suggest taking a look at `heapq` ([details](https://docs.python.org/2/library/heapq.html)), part of the Python standard library for this.  It allows you to push elemnets into a list while keeping track of the highest priority element.
```python
h = []
heappush(h, (5, 'write code'))
heappush(h, (7, 'release product'))
heappush(h, (1, 'write spec'))
heappush(h, (3, 'create tests'))
heappop(h)  // returns (1, 'write spec')
```
   
       A naive implementation would cost $O(1)$ to insert but $O(n)$ to retrieve.  `heapq` uses a [self-balancing binary search tree](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) to enable $O(\log(n))$ insertion and $O(1)$ removal. You may be asked about this data structure on an interview so it is good to get practice with it now.
    - **Working across nodes:** To obtain the largest `n`, we need to first obtain the largest n elements per chunk from the mapper, output them to the same key (reducer), and then collect the largest n elements of those in the reducer (**Question:** why does this gaurantee that we have found the largest n over the entire set?)
    - **Working within a node:** Given that we are using a priority queue, we will need to first initialize it, then `push` or `pushpop` each record to it, and finally output the top `n` after seeing each record.  For mappers, notice that these three phases correspond nicely to these three functions:
        - `mapper_init`
        - `mapper`
        - `mapper_final`

    There are similar functions in the reducer.  Also, while the run method to launch the mapreduce job is a classmethod:
        ```python
          if __name__ == '__main__':
            MRWordCount.run()
        ```
     actual instances of our mapreduce are instantiated on the map and reduce nodes.  More precisely, a separate mapper class is instantiated in each map node and a reducer class is instantiated in each reducer node.  This means that the three mapper functions can pass state through `self`, e.g. `self.heap`. Remember that to pass state between the map and reduce phase, you will have to use `yield` in the mapper and read each line in the reducer. (**Question:** Can you pass state between two mappers?)

**Checkpoint:**
- Total unique words: 1,584,646

In [152]:
f = open('output.txt','r')
output = []
for i in range(100):
    line = f.readline().split()
    word = line[0].strip('"')
    output.append((str(word),int(line[1])))

In [153]:
def top100_words_simple_plain():
    return output

grader.score(question_name='mr__top100_words_simple_plain', func=top100_words_simple_plain)

Your score:  0.96


## Question 2: top100_words_simple_text
Notice that the words "page" and "text" make it into the top 100 words in the previous problem.  These are not common English words!  If you look at the xml formatting, you'll realize that these are xml tags.  You should parse the files so that tags like `<page></page>` should not be included in your total, nor should words outside of the tag `<text></text>`.

**Hints**:
1. Both `xml.etree.elementtree` from the Python stdlib or `lxml.etree` parse xml. `lxml` is significantly faster though and avoids some bugs.

2. In order to parse the text, we will have to accumulate a `<page></page>` worth of data and then split the resulting string into words.

3. Don't forget that the Wikipedia format can have multiple revisions but you only want the latest one.

4. What happens if a content from a page is split across two different mappers? How does this problem scale with data size?

**Checkpoint:**
- Total unique words: 867,871

In [298]:
f = open('output_2.txt','r')
output_2 = []
for i in range(100):
    line = f.readline().split()
    word = line[0].strip('"')
    output_2.append((word,int(line[1])))

In [299]:
def top100_words_simple_text():
    return output_2

grader.score(question_name='mr__top100_words_simple_text', func=top100_words_simple_text)

Your score:  0.95


## Question 3: top100_words_simple_no_metadata

Finally, notice that 'www' and 'http' make it into the list of top 100 words in the previous problem.  These are also not common English words either!  These are clearly from the url in hyperlinks.  Looking at the format of [Wikipedia links](http://en.wikipedia.org/wiki/Help:Wiki_markup#Links_and_URLs) and [citations](http://en.wikipedia.org/wiki/Help:Wiki_markup#References_and_citing_sources), you'll notice that they tend to appear within single and double brackets and curly braces.

**Hint**:
You can either write a simple parser to eliminate the urls within brackets, angle braces, and curly braces or you can use a package like the colorfully-named [mwparserfromhell](https://github.com/earwig/mwparserfromhell/), which has been provisioned on `mrjob` and supports the convenient helper function `strip_code()` (which is used by the reference solution).

**Checkpoint:**
- Total unique words: 618,410

In [296]:
f = open('output_3.txt','r')
output_3 = []
for i in range(100):
    line = f.readline().split()
    word = line[0].strip('"')
    output_3.append((word,int(line[1])))

In [297]:
def top100_words_simple_no_metadata():
    return output_3

grader.score(question_name='mr__top100_words_simple_no_metadata', func=top100_words_simple_no_metadata)

Your score:  0.95


## Question 4: link_stats_simple
Let's look at some summary statistics on the number of unique links on a page to other Wikipedia articles.  Return the number of articles (count), average number of links, standard deviation, and the 25%, median, and 75% quantiles.

1. Notice that the library `mwparserfromhell` supports the method `filter_wikilinks()`.
2. You will need to compute these statistics in a way that requires O(1) memory.  You should be able to compute the first few (i.e. non-quantile) statistics exactly by looking at the first few moments of a distribution. The quantile quantities can be accurately estimated by using reservoir sampling with a large reservoir.
3. If there are multiple links to the article have it only count for 1.  This keeps our results from becoming too skewed.
4. Don't forget that some (a surprisingly large number of) links have unicode! Make sure you treat them correctly.

In [80]:
def link_stats_simple():
    return [
        ("count", 188418),
        ("mean",  15.5),
        ("stdev", 50.5),
        ("25%", 1),
        ("median", 6),
        ("75%", 16),
    ]

grader.score(question_name='mr__link_stats_simple', func=link_stats_simple)

Your score:  1.0


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*